# Carbon Use Efficiency

Resources:
* Models: http://bigg.ucsd.edu/models/

In [106]:
import requests, cobra, tempfile, os, micom, pickle
from cue_tools import loadModelURL, computeCUE, CUEDef
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Functions

In [79]:
def subsetCommunity(com, taxa):
    """
    Compute fluxes of a subset of a micom community, `com` to the list in `taxa`
    
    Discussion about approaches: 
        https://github.com/micom-dev/micom/issues/33#issuecomment-827149831
    """
    if isinstance(taxa, str):
        taxa = [taxa]
    # Remove tolerance to allow for 0 abundance
    com._rtol = 0
    # Create abundance series
    abunds = pd.Series({t: 1 if t in set(taxa) else 0 for t in com.taxa})
    com.set_abundance(abunds)

# Load

In [2]:
"""Download models locally"""
def downloadModels():
    if not os.path.isdir('models/'):
        os.makedirs('models/')
    with open('spp.txt', 'r') as f:
        IDS = f.readlines()
        for ID in IDS:
            ID = ID.strip()
            url = f'http://bigg.ucsd.edu/static/models/{ID}.mat'
            with requests.get(url) as resp:
                with open(f'models/{ID}.mat', 'wb') as o:
                    o.write(resp.content)

In [21]:
"""Setup MICOM community"""
# Don't load these models due to metabolite naming bugs
no = ['iJB785', 'iRC1080']
# Create input for MICOM
id_df = pd.read_csv('spp.txt', header=None, names=['id'])
id_df['file'] = 'models/'+id_df.id
# Filter out bad IDs
id_df = id_df[~id_df.id.isin(no)]
id_df

,id,file
0,iIT341,models/iIT341
1,iLJ478,models/iLJ478
2,iAF692,models/iAF692
3,iNF517,models/iNF517
4,iHN637,models/iHN637
6,iJN678,models/iJN678
7,iJN746,models/iJN746
8,iJR904,models/iJR904
9,iND750,models/iND750
10,iAF987,models/iAF987


In [7]:
"""Create multi-compartment 'Community' model"""
def buildCommunity(id_df):
    # ONLY FOR BUILDING COMMUNITY OBJECT, LOAD `community.pkl` directly in next cell for us
    # Build model
    com = micom.Community(id_df)
    # Pickle
    com.to_pickle('community.pkl')

In [107]:
"""Load community model"""
com = micom.load_pickle('community.pkl')
com

Name,None
Memory address,0x024515d5e108
Number of metabolites,8013
Number of reactions,9329
Number of groups,0
Objective expression,1.0*community_objective
Compartments,"c__iIT341, e__iIT341, m, c__iLJ478, e__iLJ478, c__iAF692, e__iAF692, c__iNF517, e__iNF517, e__iHN637, c__iHN637, e__iJN678, c__iJN678, u__iJN678, p__iJN678, e__iJN746, c__iJN746, p__iJN746, c__iJR904, e__iJR904, e__iND750, x__iND750, c__iND750, m__iND750, n__iND750, r__iND750, g__iND750, v__iND750, e__iAF987, c__iAF987, p__iAF987"


# Compute CUE for mono and co-cultures
Strategy: Build [micom](https://micom-dev.github.io/micom/) community model and then subset to mono and co-cultures. Building the community model once will prevent loading models for each community.

Discussion on GitHub with developer about issues: https://github.com/micom-dev/micom/issues/33#issuecomment-827149831

Assumptions:
* Even communities (50% abundance for both organisms)
* Non-fixed media: medias are model dependent and co-culture medias are a combination of medias from individual models


In [108]:
"""Compute CUE for each mono and co-culture"""
# micom.logger.logging.disable() # shhhhhh
N = len(com.taxa)
data = np.zeros((N, N))
for i in range(N):
    id1 = com.taxa[i]
    id1_data = []
    for j in range(i, N):
        id2 = com.taxa[j]
        
        # Subset community
        taxa = {id1, id2}
        subsetCommunity(com, taxa)
        
        # Compute CUE for community
        try:
            cue = CUEDef(com).loc['Def 1'].squeeze()
        except:
            cue = None
        
        # Save
        data[i, j] = data[j, i] = cue
# Ingest into df
cue_df = pd.DataFrame(data, com.taxa, com.taxa)
cue_df.to_csv('community_cue.csv')
cue_df

,iIT341,iLJ478,iAF692,iNF517,iHN637,iJN678,iJN746,iJR904,iND750,iAF987
iIT341,0.726312,NaN,0.250622,0.067122,0.051814,0.272886,0.506330,0.208087,0.103884,0.242394
iLJ478,NaN,0.211992,0.212486,0.093458,0.059052,0.233262,0.215456,0.206659,0.109393,0.207846
iAF692,0.250622,0.212486,0.226688,0.092195,0.058435,0.240648,0.230615,0.207181,0.109229,0.215045
iNF517,0.067122,0.093458,0.092195,0.034940,0.047603,0.119832,0.037039,0.191554,0.086518,0.088794
iHN637,0.051814,0.059052,0.058435,0.047603,0.048769,0.066098,0.049117,0.146907,0.069581,0.078657
iJN678,0.272886,0.233262,0.240648,0.119832,0.066098,0.258565,0.261038,0.209290,0.117573,0.236863
iJN746,0.506330,0.215456,0.230615,0.037039,0.049117,0.261038,0.655291,0.207117,0.100637,0.221057
iJR904,0.208087,0.206659,0.207181,0.191554,0.146907,0.209290,0.207117,0.206956,0.174188,0.206807
iND750,0.103884,0.109393,0.109229,0.086518,0.069581,0.117573,0.100637,0.174188,0.100192,0.108246
iAF987,0.242394,0.207846,0.215045,0.088794,0.078657,0.236863,0.221057,0.206807,0.108246,NaN
